In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer
from sklearn.neural_network import MLPRegressor 
import time

In [2]:
df = pd.read_excel(r'C:\Users\TI\Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833.0,12,0.21,2,22.0,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800.0,12,0.21,2,22.0,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826.0,12,0.21,2,22.0,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831.0,12,0.21,2,22.0,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834.0,12,0.21,2,22.0,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676.0,36,0.21,2,41.0,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683.0,36,0.21,2,42.0,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340909,50151.250000,77962.397727,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846.0,36,0.21,2,42.0,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694118,47515.850980,73948.509804,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876.0,36,0.21,2,42.0,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943182,42468.990341,65994.485795,3.789568e+07,3761.343750


In [3]:
df.dropna(inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37527 entries, 0 to 37527
Data columns (total 30 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Backhoe      37527 non-null  int64  
 1   HP(watt)     37527 non-null  float64
 2   Norm_MAP     37527 non-null  float64
 3   RPM          37527 non-null  float64
 4   Age          37527 non-null  int64  
 5   Load_Factor  37527 non-null  float64
 6   Engine_Tier  37527 non-null  int64  
 7   TEMP[C]      37527 non-null  float64
 8   Fuel[g/s]    37527 non-null  float64
 9   NOx[g/s]     37527 non-null  float64
 10  HC[g/s]      37527 non-null  float64
 11  CO[g/s]      37527 non-null  float64
 12  CO2[g/s]     37527 non-null  float64
 13  PM[mg/s]     37527 non-null  float64
 14  Nox (g/l)    37527 non-null  float64
 15  HC (g/l)     37527 non-null  float64
 16  CO (g/l)     37527 non-null  float64
 17  CO2 (g/l)    37527 non-null  float64
 18  PM (g/l)     37527 non-null  float64
 19  Fuel[g/hr

In [5]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['CO[g/s]']

In [6]:
PredictorScaler = StandardScaler()
TargetVarScaler = StandardScaler()

PredictorScalerFit = PredictorScaler.fit(X)
TargetVarScalerFit = TargetVarScaler.fit(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

X = PredictorScalerFit.transform(X)
y = TargetVarScalerFit.transform(y.values.reshape(-1, 1))  # Convert y to numpy array and reshape

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

In [8]:
mlp = MLPRegressor()
mlp

MLPRegressor()

In [9]:
start_time = time.time()
mlp.fit(X_train, y_train)
print("Execution time: " + str((time.time() - start_time)) + ' s')

Execution time: 4.364869117736816 s


In [10]:
y_pred = mlp.predict(X_test)

In [11]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f"RMSE: {rmse:.2f}")

RMSE: 0.71


In [12]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape:.2f}%")

MAPE: 2.97%


Grid

In [13]:
param = {
    'hidden_layer_sizes': range(1, 100),
    'activation':['relu','tanh', 'logistic', 'identity'],
    'alpha': np.arange(0.01, 1.0, 0.01),
    'learning_rate':['invscaling','adaptive'],
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)


Random

In [14]:
rs = RandomizedSearchCV(mlp, 
                            param, 
                            cv=3, 
                            scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
                            refit='RMSE',
                            n_iter=100,
                            verbose=1,
                            n_jobs=-1
                            )


start_time = time.time()
rs.fit(X_train, y_train)


print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Best: 0.963012 using {'learning_rate': 'invscaling', 'hidden_layer_sizes': 69, 'alpha': 0.97, 'activation': 'logistic'}
Best RMSE: 0.9630124937957733
Best MAPE: -4.977834320269571
Execution time: 71.56563806533813 s


In [15]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.9460717695708863
MAPE: 4.64%


In [16]:
pd.DataFrame(rs.cv_results_).to_excel('MLP_RS_Scal_CO.xlsx', index=False)

Bayessian

In [17]:
def mlp_bs(hidden_layer_sizes, activation, alpha, learning_rate):
    activations = ['relu','tanh', 'logistic', 'identity']
    learning_rates = ['invscaling','adaptive']
    mlpbs= MLPRegressor(
        hidden_layer_sizes=int(hidden_layer_sizes),
        activation=activations[int(activation)],
        alpha=float(alpha),
        learning_rate=learning_rates[int(learning_rate)],
    )
    scores = cross_val_score(mlpbs, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse 

In [18]:
bayes_opt = BayesianOptimization(
    mlp_bs,
    {
        'hidden_layer_sizes': (1, 100),
        'activation': (0, 3),
        'alpha': (0.01, 1.0),
        'learning_rate': (0, 1)
    }
)

In [19]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | activa... |   alpha   | hidden... | learni... |
-------------------------------------------------------------------------
| 1         | -0.9619   | 2.645     | 0.7113    | 85.38     | 0.225     |
| 2         | -0.7501   | 1.165     | 0.7746    | 29.07     | 0.7186    |


c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 3         | -0.7205   | 0.3863    | 0.03454   | 16.62     | 0.09987   |
| 4         | -0.7091   | 0.09485   | 0.2972    | 68.24     | 0.4718    |


c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 5         | -0.7563   | 1.468     | 0.598     | 9.877     | 0.5728    |


...

c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 107       | -0.7631   | 1.539     | 1.0       | 44.78     | 0.0       |
| 108       | -0.9654   | 3.0       | 0.01      | 95.2      | 0.0       |
| 109       | -0.6929   | 0.4773    | 0.01      | 88.68     | 0.0       |


c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


| 110       | -0.7309   | 0.0       | 1.0       | 91.6      | 0.0       |
Execution time: 1624.7671604156494 s


c:\Users\TI\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [20]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [21]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'activation': 3.0, 'alpha': 0.01, 'hidden_layer_sizes': 95.20425584876737, 'learning_rate': 0.0}


In [22]:
activations = ['relu','tanh', 'logistic', 'identity']
learning_rates = ['invscaling','adaptive']

best_bs = MLPRegressor(
    hidden_layer_sizes=int(best_params['hidden_layer_sizes']),
    activation=activations[int(best_params['activation'])],  # Convert int to string
    alpha=float(best_params['alpha']),
    learning_rate=learning_rates[int(best_params['learning_rate'])]  # Convert int to string
)
best_bs.fit(X_train, y_train)

MLPRegressor(activation='identity', alpha=0.01, hidden_layer_sizes=95,
             learning_rate='invscaling')

In [23]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.9446025765072337
MAPE: 4.00%
